# Frequent Working Pattern Analysis

In [21]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
from datetime import datetime, date
from datetime import datetime
import pandas as pd
import csv
from collections import defaultdict
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from dotenv import load_dotenv
import os

In [22]:
load_dotenv()
uri = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USER')
password = os.getenv('NEO4J_PASSWORD')

In [23]:
driver = GraphDatabase.driver(uri, auth=(username, password))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neo4j/_sync/driver.py:493: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  deprecation_warn(


# High-Level Event Log Construction

In [24]:
cypher_all_data = """
    MATCH (k:Kit)<-[:CORR]-(e:Event)-[:CORR]->(hbl:HighLevelBatch)
    
    WITH 
    date(e.timestamp) as date,
    e.timestamp as event_timestamp,
    e.activity as event_activity,
    e.batch as event_batch,
    k.kitId as kitId, 
    ID(hbl) as hbl_id,
    hbl.date as hbl_date,
    hbl.activity_name as hbl_activity,
    hbl.sysId as resource,
    hbl.corr_batch_numbers as hbl_corr_batch_numbers,
    hbl.workTogether as work_together
    RETURN date, event_timestamp, event_activity, event_batch, kitId, hbl_id, hbl_date, hbl_activity, resource, hbl_corr_batch_numbers, work_together

    ORDER BY date,event_timestamp, resource 
"""

def cypher_all_data_func(uri, username, password, cypher_all_data):
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run(cypher_all_data)
        data = result.data()
        return data
    
results_cypher_all_data = cypher_all_data_func(uri, username, password, cypher_all_data)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/neo4j/_sync/driver.py:493: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  deprecation_warn(


In [25]:
def convert_neo4j_date_to_python(neo4j_date):
    return date(
        year=neo4j_date.year,
        month=neo4j_date.month,
        day=neo4j_date.day
    )

def convert_neo4j_datetime_to_python(neo4j_datetime):
    return datetime(
        year=neo4j_datetime.year,
        month=neo4j_datetime.month,
        day=neo4j_datetime.day,
        hour=neo4j_datetime.hour,
        minute=neo4j_datetime.minute,
        second=neo4j_datetime.second,
        microsecond=neo4j_datetime.nanosecond // 1000, 
        tzinfo=neo4j_datetime.tzinfo
    )
df = pd.DataFrame(results_cypher_all_data)

df['date'] = df['date'].apply(convert_neo4j_date_to_python)
df['date'] = df['date'].apply(lambda d: d.strftime('%Y-%m-%d'))

df['hbl_date'] = df['hbl_date'].apply(convert_neo4j_date_to_python)
df['hbl_date'] = df['hbl_date'].apply(lambda d: d.strftime('%Y-%m-%d'))

df['event_timestamp'] = df['event_timestamp'].apply(convert_neo4j_datetime_to_python)

df['resource'] = df['resource'].astype(str)
df['work_together'] = df['work_together'].astype(str)

df['hbl_activity'] = df['hbl_activity'].astype(str)
df['hbl_activity'] = df['hbl_activity'].str.strip('[]')
df['hbl_activity'] = df['hbl_activity'].apply(lambda x: x.strip("[]").replace("'", ""))

df['hbl_corr_batch_numbers'] = df['hbl_corr_batch_numbers'].astype(str)
df['hbl_corr_batch_numbers'] = df['hbl_corr_batch_numbers'].str.strip('[]')

In [26]:
df = df.sort_values(by=['date', 'resource', 'event_timestamp'], ascending=True)

hbl_global_id = 0
previous_hbl_id = None

def assign_hbl_global_id(row):
    global hbl_global_id, previous_hbl_id
    if row['hbl_id'] != previous_hbl_id:
        hbl_global_id += 1
        previous_hbl_id = row['hbl_id']
    return hbl_global_id

df['hbl_global_id'] = df.apply(assign_hbl_global_id, axis=1)

In [27]:
grouped = df.groupby('hbl_global_id')

hlb_df = grouped.agg(
    hbl_date=('hbl_date', 'first'),
    hbl_activity=('hbl_activity', 'first'),
    resource=('resource', 'first'),
    earliest_timestamp=('event_timestamp', 'min'),
    latest_timestamp=('event_timestamp', 'max'),
    event_number=('event_timestamp', 'count'),
    kits=('kitId', 'nunique'),
    batch_instance_number=('event_batch', 'nunique'),
    batch_instances=('event_batch', lambda x: ', '.join(map(str, x.unique()))),
    hbl_id=('hbl_id', 'first'),
    work_together=('work_together', 'first')
).reset_index()

Adding additional features

In [28]:
hlb_df['hbl_date'] = pd.to_datetime(hlb_df['hbl_date'])
hlb_df['earliest_timestamp'] = pd.to_datetime(hlb_df['earliest_timestamp'])
hlb_df['latest_timestamp'] = pd.to_datetime(hlb_df['latest_timestamp'])

hlb_df['month'] = hlb_df['hbl_date'].dt.month
hlb_df['day_of_week'] = hlb_df['hbl_date'].dt.dayofweek

hlb_df = hlb_df.sort_values(by=['hbl_date', 'resource', 'hbl_global_id'])

Assigning shifts

In [29]:
def assign_shift(timestamp):
    if timestamp.time() >= pd.Timestamp('00:00:00').time() and timestamp.time() < pd.Timestamp('09:30:00').time():
        return '1'
    elif timestamp.time() >= pd.Timestamp('09:30:00').time() and timestamp.time() < pd.Timestamp('11:30:00').time():
        return '2'
    elif timestamp.time() >= pd.Timestamp('11:30:00').time() and timestamp.time() < pd.Timestamp('14:00:00').time():
        return '3'
    else:
        return '4'

hlb_df['shift'] = hlb_df.groupby(['resource', 'hbl_date'])['earliest_timestamp'].transform(lambda x: assign_shift(x.min()))
hlb_df['shift'] = hlb_df['shift'].astype(int)

In [30]:
final_df_weekdays = hlb_df[hlb_df['day_of_week'] < 5]
final_df_weekends = hlb_df[hlb_df['day_of_week'] > 4]

# Frequent Patterns Weekdays 

In [31]:
final_df_weekdays['earliest_timestamp'] = pd.to_datetime(final_df_weekdays['earliest_timestamp'])
final_df_weekdays['latest_timestamp'] = pd.to_datetime(final_df_weekdays['latest_timestamp'])
final_df_weekdays['work_together'] = final_df_weekdays['work_together'].astype(str)

def extract_sequences(df):
    sequences = defaultdict(list)
    for _, group in df.groupby(['resource', 'day_of_week', 'shift', 'hbl_date']):
        seq = []
        for _, row in group.sort_values('earliest_timestamp').iterrows():
            activity = f"{row['hbl_activity']} ({'Together' if row['work_together'] == 'True' else 'Separate'})"
            seq.append(activity)
        if len(seq) > 1: 
            sequences[(group['resource'].iloc[0], group['day_of_week'].iloc[0], group['shift'].iloc[0], group['hbl_date'].iloc[0])] = seq
    return sequences

sequences = extract_sequences(final_df_weekdays)

flat_sequences = list(sequences.values())

# Apply TransactionEncoder to transform the sequences for pattern mining
te = TransactionEncoder()
te_ary = te.fit(flat_sequences).transform(flat_sequences, sparse=True)
df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

# Apply the fpgrowth algorithm to find frequent patterns
frequent_patterns = fpgrowth(df_encoded, min_support=0.1, use_colnames=True)

frequent_patterns['sequence'] = frequent_patterns['itemsets'].apply(lambda x: ' -> '.join(list(x)))
frequent_patterns = frequent_patterns.drop(columns=['itemsets'])

context_records = []
for (resource, day_of_week, shift, hbl_date), seq in sequences.items():
    context_records.append({
        'resource': resource, 
        'day_of_week': day_of_week, 
        'shift': shift, 
        'hbl_date': hbl_date,
        'sequence': seq,
        'sequence_str': ' -> '.join(seq)
    })
context_df = pd.DataFrame(context_records)

total_sequences = context_df.groupby(['day_of_week', 'shift']).size().reset_index(name='Traces on a weekday and shift')

def match_patterns(patterns_df, context_df):
    matched_patterns = []
    for _, pattern_row in patterns_df.iterrows():
        pattern = pattern_row['sequence']
        pattern_parts = pattern.split(' -> ')
        for _, context_row in context_df.iterrows():
            context_sequence = context_row['sequence']
            context_sequence_str = context_row['sequence_str']
            if all(item in context_sequence_str for item in pattern_parts):
                pattern_work_together = [part.split('(')[1].strip(')') for part in pattern_parts]
                context_work_together = [activity.split('(')[1].strip(')') for activity in context_sequence if any(activity.startswith(part.split(' (')[0]) for part in pattern_parts)]
                if pattern_work_together == context_work_together:
                    matched_patterns.append({
                        'day_of_week': context_row['day_of_week'],
                        'shift': context_row['shift'],
                        'sequence': pattern,
                        'resource': context_row['resource'],
                        'hbl_date': context_row['hbl_date']
                    })
    return pd.DataFrame(matched_patterns)


matched_patterns_df = match_patterns(frequent_patterns, context_df)

matched_patterns_df_unique = matched_patterns_df.drop_duplicates(subset=['day_of_week', 'shift', 'sequence', 'resource', 'hbl_date'])

pattern_counts = matched_patterns_df_unique.groupby(['day_of_week', 'shift', 'sequence']).size().reset_index(name='Traces with a pattern')

pattern_counts = pattern_counts.merge(total_sequences, on=['day_of_week', 'shift'])
pattern_counts['Percentage'] = (pattern_counts['Traces with a pattern'] / pattern_counts['Traces on a weekday and shift']) * 100

day_mapping = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
pattern_counts['Day name'] = pattern_counts['day_of_week'].map(day_mapping)

shift_order = [1, 2, 3, 4]
pattern_counts['shift'] = pd.Categorical(pattern_counts['shift'], categories=shift_order, ordered=True)

/var/folders/1r/3fk_x7p96299qsbl_p0t7v6h0000gn/T/ipykernel_14691/3721520294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_weekdays['earliest_timestamp'] = pd.to_datetime(final_df_weekdays['earliest_timestamp'])
/var/folders/1r/3fk_x7p96299qsbl_p0t7v6h0000gn/T/ipykernel_14691/3721520294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_weekdays['latest_timestamp'] = pd.to_datetime(final_df_weekdays['latest_timestamp'])
/var/folders/1r/3fk_x7p96299qsbl_p0t7v6h0000gn/T/ipykernel_14691

# Frequent Patterns Weekends

In [33]:
final_df_weekends['earliest_timestamp'] = pd.to_datetime(final_df_weekends['earliest_timestamp'])
final_df_weekends['latest_timestamp'] = pd.to_datetime(final_df_weekends['latest_timestamp'])
final_df_weekends['work_together'] = final_df_weekends['work_together'].astype(str)

def extract_sequences(df):
    sequences = defaultdict(list)
    for _, group in df.groupby(['resource', 'day_of_week', 'shift', 'hbl_date']):
        seq = []
        for _, row in group.sort_values('earliest_timestamp').iterrows():
            activity = f"{row['hbl_activity']} ({'Together' if row['work_together'] == 'True' else 'Separate'})"
            seq.append(activity)
        if len(seq) > 1: 
            sequences[(group['resource'].iloc[0], group['day_of_week'].iloc[0], group['shift'].iloc[0], group['hbl_date'].iloc[0])] = seq
    return sequences

sequences_weekends = extract_sequences(final_df_weekends)

flat_sequences_weekends = list(sequences_weekends.values())

# Apply TransactionEncoder to transform the sequences for pattern mining
te = TransactionEncoder()
te_ary = te.fit(flat_sequences_weekends).transform(flat_sequences_weekends, sparse=True)
df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

# Apply the fpgrowth algorithm to find frequent patterns
frequent_patterns_weekends = fpgrowth(df_encoded, min_support=0.1, use_colnames=True)

frequent_patterns_weekends['sequence'] = frequent_patterns_weekends['itemsets'].apply(lambda x: ' -> '.join(list(x)))
frequent_patterns_weekends = frequent_patterns_weekends.drop(columns=['itemsets'])

context_records = []
for (resource, day_of_week, shift, hbl_date), seq in sequences_weekends.items():
    context_records.append({
        'resource': resource, 
        'day_of_week': day_of_week, 
        'shift': shift, 
        'hbl_date': hbl_date,
        'sequence': seq,
        'sequence_str': ' -> '.join(seq)
    })
context_df = pd.DataFrame(context_records)

total_sequences_weekends = context_df.groupby(['day_of_week', 'shift']).size().reset_index(name='Traces on a weekday and shift')

def match_patterns(patterns_df, context_df):
    matched_patterns = []
    for _, pattern_row in patterns_df.iterrows():
        pattern = pattern_row['sequence']
        pattern_parts = pattern.split(' -> ')
        for _, context_row in context_df.iterrows():
            context_sequence = context_row['sequence']
            context_sequence_str = context_row['sequence_str']
            if all(item in context_sequence_str for item in pattern_parts):
                pattern_work_together = [part.split('(')[1].strip(')') for part in pattern_parts]
                context_work_together = [activity.split('(')[1].strip(')') for activity in context_sequence if any(activity.startswith(part.split(' (')[0]) for part in pattern_parts)]
                if pattern_work_together == context_work_together:
                    matched_patterns.append({
                        'day_of_week': context_row['day_of_week'],
                        'shift': context_row['shift'],
                        'sequence': pattern,
                        'resource': context_row['resource'],
                        'hbl_date': context_row['hbl_date']
                    })
    return pd.DataFrame(matched_patterns)

matched_patterns_df_weekends = match_patterns(frequent_patterns_weekends, context_df)

matched_patterns_df_unique_weekends = matched_patterns_df_weekends.drop_duplicates(subset=['day_of_week', 'shift', 'sequence', 'resource', 'hbl_date'])

pattern_counts_weekends = matched_patterns_df_unique_weekends.groupby(['day_of_week', 'shift', 'sequence']).size().reset_index(name='Traces with a pattern')

pattern_counts_weekends = pattern_counts_weekends.merge(total_sequences_weekends, on=['day_of_week', 'shift'])
pattern_counts_weekends['Percentage'] = (pattern_counts_weekends['Traces with a pattern'] / pattern_counts_weekends['Traces on a weekday and shift']) * 100

day_mapping = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
pattern_counts_weekends['Day name'] = pattern_counts_weekends['day_of_week'].map(day_mapping)

shift_order = [1, 2, 3, 4]
pattern_counts_weekends['shift'] = pd.Categorical(pattern_counts_weekends['shift'], categories=shift_order, ordered=True)


/var/folders/1r/3fk_x7p96299qsbl_p0t7v6h0000gn/T/ipykernel_14691/778533589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_weekends['earliest_timestamp'] = pd.to_datetime(final_df_weekends['earliest_timestamp'])
/var/folders/1r/3fk_x7p96299qsbl_p0t7v6h0000gn/T/ipykernel_14691/778533589.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_weekends['latest_timestamp'] = pd.to_datetime(final_df_weekends['latest_timestamp'])
/var/folders/1r/3fk_x7p96299qsbl_p0t7v6h0000gn/T/ipykernel_14691/7